In [1]:
from pyspark import SparkConf , SparkContext

conf = SparkConf().setMaster("local").setAppName("mnm-rdd-counts")
spark = SparkContext(conf=conf)

24/12/06 11:21:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os 
directory = os.path.join(os.getcwd(),"data")
filename = "mnm_dataset.csv"
filepath = os.path.join(directory, filename)

In [3]:
#RDD 생성
lines = spark.textFile("file:///"+filepath.replace("ww", "/"))
lines 

file:////home/lab09/src/data/mnm_dataset.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [4]:
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [5]:
header = lines.first()
datas = lines.filter(lambda row : row != header)

In [6]:
datas.take(10)

['TX,Red,20',
 'NV,Blue,66',
 'CO,Blue,79',
 'OR,Blue,71',
 'WA,Yellow,93',
 'WY,Blue,16',
 'CA,Yellow,53',
 'WA,Green,60',
 'OR,Green,71',
 'TX,Green,68']

In [7]:
# 1. 데이터 RDD 생성 map()
def parse(row):
    fields = row.split(",")
    state = fields[0]
    color = fields[1]
    count = int(fields[2])
    
    return color, count

In [8]:
color_counts = datas.map(parse)
color_counts.take(10)

[('Red', 20),
 ('Blue', 66),
 ('Blue', 79),
 ('Blue', 71),
 ('Yellow', 93),
 ('Blue', 16),
 ('Yellow', 53),
 ('Green', 60),
 ('Green', 71),
 ('Green', 68)]

In [9]:
color_counts = color_counts.mapValues(lambda x : (x, 1))
color_counts.take(10)

[('Red', (20, 1)),
 ('Blue', (66, 1)),
 ('Blue', (79, 1)),
 ('Blue', (71, 1)),
 ('Yellow', (93, 1)),
 ('Blue', (16, 1)),
 ('Yellow', (53, 1)),
 ('Green', (60, 1)),
 ('Green', (71, 1)),
 ('Green', (68, 1))]

In [10]:
# 2. collect()
result = color_counts.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
result = result.collect()
result

[('Red', (916864, 16619)),
 ('Blue', (902516, 16449)),
 ('Yellow', (923009, 16796)),
 ('Green', (926150, 16928)),
 ('Brown', (915703, 16510)),
 ('Orange', (915793, 16697))]

In [11]:
# 3. blue mnms만 꺼내기
result[1]

('Blue', (902516, 16449))

In [12]:
# 4. 전체 합계 구하기
total_counts = sum(row[1][0] for row in result)
total_counts 

5500035

In [13]:
# 5. 도시별 합계 구하기

def CityParse(row):
    fields = row.split(",")
    state = fields[0]
    count = int(fields[2])
    return state, count

In [14]:
city_counts = datas.map(CityParse)
city_counts=city_counts.mapValues(lambda x : (x,1))
city_counts.take(10)

[('TX', (20, 1)),
 ('NV', (66, 1)),
 ('CO', (79, 1)),
 ('OR', (71, 1)),
 ('WA', (93, 1)),
 ('WY', (16, 1)),
 ('CA', (53, 1)),
 ('WA', (60, 1)),
 ('OR', (71, 1)),
 ('TX', (68, 1))]

In [94]:
city_counts = city_counts.reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
city_counts.collect()

[('TX', (556493, 10072)),
 ('NV', (548477, 10025)),
 ('CO', (556302, 10051)),
 ('OR', (542169, 9903)),
 ('WA', (557227, 10065)),
 ('WY', (538975, 9782)),
 ('CA', (561184, 10164)),
 ('AZ', (546812, 10001)),
 ('NM', (553454, 10050)),
 ('UT', (538942, 9886))]

In [15]:
spark.stop()